
# Phase 5c – ColabFold Multimer Validation

This notebook runs **ColabFold Multimer** to validate:

1. Inter-chain salt bridges  
2. a/d core packing distances  
3. True helix crossing angle  

Upload your FASTA file (`gcn4_all_multimer.fasta`) before running.


In [ ]:

# Install ColabFold (Google Colab only)
!pip install -q colabfold
!pip install -q biopython numpy scipy


## Run ColabFold Multimer

In [ ]:

from colabfold.batch import run
from pathlib import Path

fasta_path = "gcn4_all_multimer.fasta"  # Upload this file
output_dir = "colabfold_output"

run(
    fasta_path,
    output_dir,
    model_type="alphafold2_multimer_v3",
    num_models=5,
    num_recycles=3,
    amber=False
)

print("ColabFold run complete.")


## Salt Bridge Analysis

In [ ]:

from Bio.PDB import PDBParser
import numpy as np

def detect_salt_bridges(pdb_file, cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('model', pdb_file)
    chains = list(structure.get_chains())
    if len(chains) < 2:
        return 0
    
    chain_A = list(chains[0].get_residues())
    chain_B = list(chains[1].get_residues())
    
    salt_bridges = 0
    
    for resA in chain_A:
        for resB in chain_B:
            if resA.get_resname() in ["GLU","ASP"] and resB.get_resname() in ["LYS","ARG"]:
                for atomA in resA:
                    for atomB in resB:
                        if atomA - atomB < cutoff:
                            salt_bridges += 1
                            break
    return salt_bridges

print("Function loaded.")


## Core Packing (Cβ–Cβ distance)

In [ ]:

def cb_distance(resA, resB):
    if 'CB' in resA and 'CB' in resB:
        return resA['CB'] - resB['CB']
    return None

print("Cβ distance function ready.")


## Helix Crossing Angle

In [ ]:

def fit_axis(coords):
    coords = np.array(coords)
    centroid = coords.mean(axis=0)
    u, s, vh = np.linalg.svd(coords - centroid)
    return vh[0]

print("Axis fitting ready.")
